In [1]:
%matplotlib inline
from __future__ import division
import pandas as pd

In [2]:
def digit_precision(value, precision=10):
    print ("%.{}f".format(precision-len(str(int(value)))) % value)

In [3]:
import requests
import os

month = ["%.2d" % i for i in range(1, 13)]
print ("Month of the year: ", month)

lst_df = []
for m in month: 
    url = 'https://s3.amazonaws.com/tripdata/2015{}-citibike-tripdata.zip'.format(m)
    r = requests.get(url)
    with open("citibike2015{}.zip".format(m), "wb") as code:
        code.write(r.content)
    print("Completed Download {}".format(m))
    lst_df.append(pd.read_csv("citibike2015{}.zip".format(m), compression = 'zip')) 
df = pd.concat(lst_df, axis=0, keys=range(1, 13))
df.to_csv(r'citibike2015All.csv')

filelist = [ f for f in os.listdir(".") if f.endswith(".zip") ]
for f in filelist:
    os.remove(f)
    
print("Successfully removed zip files")


Month of the year:  ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
Completed Download 01
Completed Download 02
Completed Download 03
Completed Download 04
Completed Download 05
Completed Download 06
Completed Download 07
Completed Download 08
Completed Download 09
Completed Download 10
Completed Download 11
Completed Download 12
Successfully removed zip files


In [4]:
sd

NameError: name 'sd' is not defined

In [ ]:
# ## !! For Subquestions 1-4 timestamps are not required, 
# # Therefore dataframe can be loaded faster from csv and parse_dates be ommited !!!

# df = pd.read_csv(r'citibike2015All.csv', index_col=[0, 1], error_bad_lines=False)
# #, parse_dates=[3, 4])
# df.index.names = [None, None]

In [ ]:
df.head()

#### 1. What is the median trip duration, in seconds? 

In [ ]:
digit_precision(df.tripduration.median())

#### 2. What fraction of rides start and end at the same station? 

In [ ]:
digit_precision(len(df[df['start station id'] == df['end station id']])/len(df))

#### 3. We say a bike has visited a station if it has a ride that either started or ended at that station. Some bikes have visited many stations; others just a few. What is the standard deviation of the number of stations visited by a bike? 

In [ ]:
grouped = df.groupby('bikeid')

In [ ]:
bikeid_df  = pd.DataFrame()
bikeid_df['start'] =  grouped['start station id'].unique()
bikeid_df['end'] =  grouped['end station id'].unique()

In [ ]:
bikeid_df.head()

In [ ]:
# Join the two sets (do not count stations double)
bikeid_df['union'] = bikeid_df.apply(lambda x: set(x['start']) | set(x['end']), axis=1)

In [ ]:
bikeid_df.head()

In [ ]:
# Count number of elements in set
bikeid_df['stations_visited'] = bikeid_df['union'].apply(lambda x: len(x))

In [ ]:
bikeid_df.head(3)

In [ ]:
# Use pd describe to show standard deviation!!
print (digit_precision(bikeid_df['stations_visited'].std()))
bikeid_df['stations_visited'].hist();

In [ ]:
# Delete df to free up memory
del bikeid_df

#### 4. What is the average length, in kilometers, of a trip? Assume trips follow great circle arcs from the start station to the end station. Ignore trips that start and end at the same station, as well as those with obviously wrong data. 

In [ ]:
# Use geopy distance to calculate great_circle (return km)
from geopy.distance import great_circle
df['start_location'] = list(zip(df['start station latitude'],df['start station longitude']))
df['end_location'] = list(zip(df['end station latitude'],df['end station longitude']))

In [ ]:
x = df.iloc[0]
print (x['start_location'], x['end_location'])

In [ ]:
x

In [ ]:
df['trip_length'] = df.apply(lambda x: great_circle(x['start_location'], x['end_location']).km, axis=1)

Questions explicitly asks to check for obviously wrong data, 

In [ ]:
print ('Example for wrong birth date give', df.ix[df['birth year'].idxmin()])
print ('Number of missing birth years ', len(df[df['birth year'].isnull()]))

- Many users have provided no or wrong data about their birth year
- Nevertheless, with respect to the average lenght of the trip this was assumed to be irrelevant

In [ ]:
# Compute average speed in km/h (convert seconds to hours division by 3600)
df['avg_speed_km/h'] = df['trip_length'] / (df['tripduration'] / 3600)

In [ ]:
# Show row with highest average speed, that is an obviously wrong data point
df.ix[df['avg_speed_km/h'].idxmax()]

In [ ]:
# Asssume that the maximal average speed per trip is 30 km/h, Show how many data points are removed
len(df[df['avg_speed_km/h']>=30])

In [ ]:
len(df[df['avg_speed_km/h']==0])

In [ ]:
# Remove trips where either the trip starts and ends at the same station and where the average speed for the trip was > 30km/h
cleaned_df = df[~(df['avg_speed_km/h']==0) | (df['avg_speed_km/h']>=30)]

In [ ]:
# Show end result, average lenght in kilometers
print cleaned_df['trip_length'].mean()
df['trip_length'].hist();

-------------------------------------------------------------------------------------------------------------------------------